In [2]:
import glob
import pathlib
import pandas as pd
import cv2
import numpy as np

rpi = 'h2r4'

path_to_folder = '../live_bees/' + rpi + '/random_images_25/*.jpg'
folder = sorted(glob.glob(path_to_folder))
names = [pathlib.Path(x).stem for x in folder]

path_to_mask = '../live_bees/' + rpi + '/masks/*.csv'
masks = sorted(glob.glob(path_to_mask))

dfs = []
for mask in masks:
    df = pd.read_csv(mask)
    dfs.append(df)

print(len(folder))
print(len(masks))

21
21


In [3]:
x_max = 2592
y_max = 4608
imgs = []

for df, file in zip(dfs, folder):
    img = cv2.imread(file)
    img = cv2.resize(img, (4608, 2592))
    for nbr_of_clusters in df['index'].unique():
        points = []
        df_bis = df[df['index'] == nbr_of_clusters]
        for x, y in zip(df_bis['axis-0'], df_bis['axis-1']):
            if x < 0: x = 0
            elif x > x_max: x = x_max
            if y < 0: y = 0
            elif y > y_max: y = y_max
            points.append((int(y), int(x)))
            cv2.circle(img, (int(y), int(x)), 20, (0, 255, 0), -1)
        ctr = np.array(points).reshape((-1,1,2)).astype(np.int32)
        cv2.drawContours(img, [ctr], -1, (255, 0, 0), 5)
    imgs.append(img)

print(len(imgs))

21


In [4]:
# save all imgs 
save_folder  = '../annotated_images/' + rpi + '/'
for i, img in enumerate(imgs):
    cv2.imwrite(save_folder + names[i] + '.jpg', img)